In [1]:
import torch
from transformers import AutoTokenizer
from transformers import TextGenerationPipeline, AutoModelForCausalLM, LlamaTokenizerFast, AutoModelForSequenceClassification
import os
os.environ["WANDB_DISABLED"]="true"

In [2]:
model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-uncased", trust_remote_code=True, num_labels=105).cuda()
tokenizer: LlamaTokenizerFast = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
# from accelerate import Accelerator
import pandas as pd
import os
import numpy as np
import gc
import copy
from tqdm import tqdm
import transformers
from typing import Dict, Optional, Sequence

IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = "[PAD]"
DEFAULT_EOS_TOKEN = "</s>"
DEFAULT_BOS_TOKEN = "<s>"
DEFAULT_UNK_TOKEN = "<unk>"

def smart_tokenizer_and_embedding_resize(
    special_tokens_dict: Dict,
    tokenizer: transformers.PreTrainedTokenizer,
    model: transformers.PreTrainedModel,
):
    """Resize tokenizer and embedding.

    Note: This is the unoptimized version that may make your embedding size not be divisible by 64.
    """
    num_new_tokens = tokenizer.add_special_tokens(special_tokens_dict)
    model.resize_token_embeddings(len(tokenizer))

    if num_new_tokens > 0:
        input_embeddings = model.get_input_embeddings().weight.data
        # output_embeddings = model.get_output_embeddings().weight.data

        input_embeddings_avg = input_embeddings[:-num_new_tokens].mean(dim=0, keepdim=True)
        # output_embeddings_avg = output_embeddings[:-num_new_tokens].mean(dim=0, keepdim=True)

        input_embeddings[-num_new_tokens:] = input_embeddings_avg
        # output_embeddings[-num_new_tokens:] = output_embeddings_avg

def get_special_tokens_dict(tokenizer):
    special_tokens_dict = dict()
    if tokenizer.pad_token is None:
        special_tokens_dict["pad_token"] = DEFAULT_PAD_TOKEN
    if tokenizer.eos_token is None:
        special_tokens_dict["eos_token"] = DEFAULT_EOS_TOKEN
    if tokenizer.bos_token is None:
        special_tokens_dict["bos_token"] = DEFAULT_BOS_TOKEN
    if tokenizer.unk_token is None:
        special_tokens_dict["unk_token"] = DEFAULT_UNK_TOKEN
    return special_tokens_dict

In [4]:
special_tokens_dict = get_special_tokens_dict(tokenizer)
smart_tokenizer_and_embedding_resize(
    special_tokens_dict=special_tokens_dict,
    tokenizer=tokenizer,
    model=model,
)
model.config.pad_token_id = tokenizer.pad_token_id

In [6]:
# print(data.columns)
# print(len(data))
# print(data.iloc[0])
# for d in data.iloc[0]:
#     print(type(d))

In [7]:
# import pandas as pd

# # 假设data是一个pandas DataFrame，并且已经加载了相应的数据
# # 统计Label列中不同标签的数量
# label_counts = data['Label'].value_counts()
# # 获取不同标签的列表
# unique_labels = label_counts.index.tolist()
# print(unique_labels)
# print(len(unique_labels))

In [10]:
# import pandas as pd

# # 假设 data 是一个已经存在的 pandas DataFrame

# # 检查 'TextData' 列是否全为非空字符串
# all_strings = data['TextData'].apply(lambda x: isinstance(x, str) and x.strip() != '').all()

# # 检查 'LabelIndex' 列是否都为 int
# all_ints = data['LabelIndex'].apply(lambda x: isinstance(x, int) and not pd.isnull(x)).all()

# # 打印结果
# print("'TextData' 是否全为非空字符串:", all_strings)
# print("'LabelIndex' 是否都为 int:", all_ints)

In [5]:
from transformers import Trainer, TrainingArguments, AutoTokenizer, AutoModelForSequenceClassification, EvalPrediction
import torch
from sklearn.metrics import accuracy_score, recall_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    logits, labels = torch.tensor(logits), torch.tensor(labels)
    predictions = torch.argmax(logits, dim=-1)
    accuracy = accuracy_score(labels, predictions)
    recall = recall_score(labels, predictions, zero_division=0, average="weighted")
    return {'accuracy': accuracy, 'recall': recall}

In [6]:
from transformers import Trainer, TrainingArguments, AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
import pandas as pd

# 定义训练参数
training_args = TrainingArguments(
    output_dir='/opt/tiger/network/results',          # 输出目录
    num_train_epochs=50,              # 训练轮数
    per_device_train_batch_size=32,  # 训练时每个设备的batch size
    per_device_eval_batch_size=800,   # 评估时的batch size
    warmup_steps=500,                # 预热步数
    weight_decay=0.01,               # 权重衰减
    logging_dir='./logs',            # 日志目录
    logging_steps=10,
    learning_rate=1e-5,
    evaluation_strategy="no",
    save_strategy='steps',
    save_steps=500,
    save_total_limit=3
)

# 定义数据处理器
class CustomDataset:
    def __init__(self, texts, labels, tokenizer):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        inputs = self.tokenizer(text, padding='max_length', truncation=True, max_length=512, return_tensors='pt')
        return {
            'input_ids': inputs['input_ids'].squeeze(),  # Remove batch dimension
            'attention_mask': inputs['attention_mask'].squeeze(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [7]:
import pandas as pd
data = pd.read_csv("/opt/tiger/network_attck/Phishing_Email.csv")
print(data.columns)
# 使用 apply 函数和 lambda 表达式对每一行进行处理
# join 方法将列值通过空格连接成一个字符串
# data['combined_columns'] = data.drop(columns=['Email Type']).apply(lambda row: ' '.join(row.astype(str)), axis=1)

# 查看结果
print(data['Email Text'][0])
data = data[data['Email Text'].apply(lambda x: isinstance(x, str))]
print(len(data))

Index(['Unnamed: 0', 'Email Text', 'Email Type'], dtype='object')
re : 6 . 1100 , disc : uniformitarianism , re : 1086 ; sex / lang dick hudson 's observations on us use of 's on ' but not 'd aughter ' as a vocative are very thought-provoking , but i am not sure that it is fair to attribute this to " sons " being " treated like senior relatives " . for one thing , we do n't normally use ' brother ' in this way any more than we do 'd aughter ' , and it is hard to imagine a natural class comprising senior relatives and 's on ' but excluding ' brother ' . for another , there seem to me to be differences here . if i am not imagining a distinction that is not there , it seems to me that the senior relative terms are used in a wider variety of contexts , e . g . , calling out from a distance to get someone 's attention , and hence at the beginning of an utterance , whereas 's on ' seems more natural in utterances like ' yes , son ' , ' hand me that , son ' than in ones like ' son ! ' or ' so

In [8]:
# 将Label列转换为整数索引
data['LabelIndex'], unique_labels = pd.factorize(data['Email Type'])
print(unique_labels)
# data.to_csv("processed_ddos_csv.csv")

Index(['Safe Email', 'Phishing Email'], dtype='object')


In [9]:
# 准备数据集
texts = data['Email Text'].tolist()  # 用实际的文本列名替换'YourTextColumn'
labels = data['LabelIndex'].tolist()

# 划分训练集和验证集
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2)

# 创建数据集实例
train_dataset = CustomDataset(train_texts, train_labels, tokenizer)
test_dataset = CustomDataset(test_texts, test_labels, tokenizer)

In [15]:
import torch

# 保存 train_dataset
torch.save({
    'texts': train_texts,
    'labels': train_labels,
}, '/opt/tiger/network_attck/fish_train_dataset.pt')

# 保存 test_dataset
torch.save({
    'texts': test_texts,
    'labels': test_labels,
}, '/opt/tiger/network_attck/fish_test_dataset.pt')

In [5]:
train, test = torch.load("train_dataset.pt"), torch.load("test_dataset.pt")
train_texts, train_labels = train["texts"], train["labels"]
test_texts, test_labels = test["texts"], test["labels"]
train_dataset = CustomDataset(train_texts, train_labels, tokenizer)
test_dataset = CustomDataset(test_texts, test_labels, tokenizer)

In [10]:
# 初始化Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,  # 传入测试数据集
    compute_metrics=compute_metrics,  # 指定计算指标的函数
)

Detected kernel version 5.4.143, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [11]:
# 开始训练
trainer.train()

n192-024-074:365874:365874 [0] NCCL INFO cudaDriverVersion 12010
n192-024-074:365874:365874 [0] NCCL INFO NCCL_SOCKET_FAMILY set by environment to AF_INET6
n192-024-074:365874:365874 [0] NCCL INFO NCCL_SOCKET_IFNAME set by environment to eth0
n192-024-074:365874:365874 [0] NCCL INFO Bootstrap : Using eth0:fdbd:dc61:7:34::74<0>
n192-024-074:365874:365874 [0] NCCL INFO NET/Plugin : dlerror=libnccl-net.so: cannot open shared object file: No such file or directory No plugin found (libnccl-net.so), using internal implementation
NCCL version 2.20.5+cuda12.4
n192-024-074:365874:366392 [1] NCCL INFO NCCL_IB_DISABLE set by environment to 0.
n192-024-074:365874:366392 [1] NCCL INFO NCCL_SOCKET_FAMILY set by environment to AF_INET6
n192-024-074:365874:366392 [1] NCCL INFO NCCL_SOCKET_IFNAME set by environment to eth0
n192-024-074:365874:366392 [1] NCCL INFO NCCL_IB_HCA set to mlx5_2:1
n192-024-074:365874:366392 [1] NCCL INFO NET/IB : Using [0]mlx5_2:1/RoCE [RO]; OOB eth0:fdbd:dc61:7:34::74<0>
n19

/home/tiger/.local/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


/home/tiger/.local/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


KeyboardInterrupt: 

In [12]:
trainer.evaluate()

{'eval_loss': 0.06598091870546341,
 'eval_accuracy': 0.9822913871746713,
 'eval_recall': 0.9822913871746713}